In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

#Kfold no. 1, 2, 3, 4 only
num = 

#Dataset. "YoutubeDataset10" or "CustomDataset10." without the quotation marks("")
dataset = 

model = 'efficientNetV2S+LSTM'

train_path = f'/{dataset}_TRAIN_KFOLD{num}'
val_path = f'/{dataset}_VAL_KFOLD{num}'

train_dataset_path = os.listdir(train_path)
val_dataset_path = os.listdir(val_path)

In [ ]:
rooms = []

for item in train_dataset_path:
 # Get all the file names
 all_rooms = os.listdir(train_path + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str(train_path + '/' +item) + '/' + room))
    
# Build a dataframe        
train_dataframe = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_dataframe.head())
print(train_dataframe.tail())

In [ ]:
train_df = train_dataframe.loc[:,['video_name','tag']]
train_df
train_df.to_csv(f'youtube_{model}_train_dataframe_KFOLD{num}.csv')

In [ ]:
rooms = []

for item in val_dataset_path:
 # Get all the file names
 all_rooms = os.listdir(val_path + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str(val_path + '/' +item) + '/' + room))
    
# Build a dataframe        
val_dataframe = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(val_dataframe.head())
print(val_dataframe.tail())

In [ ]:
val_df = val_dataframe.loc[:,['video_name','tag']]
val_df
val_df.to_csv(f'youtube_{model}_val_dataframe_KFOLD{num}.csv')

In [ ]:
from tensorflow import keras
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8120)])
  except RuntimeError as e:
    print(e)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
print(tf.version.VERSION)

In [ ]:
import sys
sys.version

In [ ]:
train_df = pd.read_csv(f'youtube_{model}_train_dataframe_KFOLD{num}.csv')


print(f"Total videos in train_df: {len(train_df)}")



train_df.sample(10)

In [ ]:
val_df = pd.read_csv(f'youtube_{model}_val_dataframe_KFOLD{num}.csv')


print(f"Total videos in val_df: {len(val_df)}")



val_df.sample(10)

In [ ]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
#APPLY YOUR OWN PREPROCESSING TECHNIQUE AS NEED BE
#CROPPING SPECIFICATIONS IS SPECIFIC FOR CustomDataset10
IMG_SIZE = 448
NUM_FRAMES = 30
NUM_CHANNELS = 3


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=NUM_FRAMES, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)         
            frame = cv2.resize(frame, resize)
            frame = frame[112:336, 112:336]
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            frame = frame[ :, :, [2, 1, 0]]
            frames.append(frame)
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
#train
label_processor_train = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
labels_train = train_df["tag"].values
labels_train = label_processor_train(labels_train[..., None]).numpy()
print(label_processor_train.get_vocabulary())

In [ ]:
#val
label_processor_val = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(val_df["tag"]))
labels_val = val_df["tag"].values
labels_val = label_processor_val(labels_val[..., None]).numpy()
print(label_processor_val.get_vocabulary())

In [ ]:
from sklearn.utils import shuffle
train_df = shuffle(train_df)
val_df = shuffle(val_df)

In [ ]:
i3d_input_size = 224

def build_feature_extractor():
    feature_extractor = tf.keras.applications.EfficientNetV2S(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(i3d_input_size, i3d_input_size, 3),
    )
    preprocess_input = tf.keras.applications.efficientnet_v2.preprocess_input

    inputs = keras.Input((i3d_input_size, i3d_input_size, 3))
    preprocessed = preprocess_input(inputs)
    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

feature_extractor = build_feature_extractor()

In [ ]:
MAX_SEQ_LENGTH = 30
NUM_FEATURES = 2048

In [ ]:
def prepare_all_videos(df):
  
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor_train(labels[..., None]).numpy()
    print(len(labels))

    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        print("Video number: ", idx)
        print("Path:         ", path)
        # Gather all its frames and add a batch dimension.
        frames = load_video(path)
        frames = frames[None, ...]
        # Initialize placeholders to store the masks and features of the current video.
        #temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            print("length:  \n", length)
            
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :], verbose=0
                )
        frame_features[idx,] = temp_frame_features.squeeze()
        
    return frame_features, labels

In [ ]:
train_data, train_labels = prepare_all_videos(train_df)

In [ ]:
val_data, val_labels = prepare_all_videos(val_df)

In [ ]:
print(f"Train Data Shape: {train_data.shape}")
print(f"Train Labels Shape: {train_labels.shape}")
print(f"Val Data Shape: {val_data.shape}")
print(f"Val Labels Shape: {val_labels.shape}")

In [ ]:
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=False)

# LearningRate = InitialLearningRate * Droprate ^ (Epoch/EpochDrop)
# LearningRate = initialLearningRate * decay_rate ^ (step/decay_steps)
# step is best if it is proportionate with the global_step of the algorithm

In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor_train.get_vocabulary()
    len(class_vocab)
    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))

    x = keras.layers.LSTM(512)(
        frame_features_input
    )    
    x = keras.layers.Dropout(0.25)(x)
    x = keras.layers.Dense(1024, activation="relu")(x)
    x = keras.layers.Dropout(0.25)(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)
    rnn_model = keras.Model(frame_features_input, output)
    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics=["accuracy"]
    )
    return rnn_model
seq_model = get_sequence_model()
seq_model.summary()


In [ ]:
# Utility for running experiments.

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
)

callbacks = [early_stopping]

seq_model = get_sequence_model()
history = seq_model.fit(
    train_data,
    train_labels,
    validation_data = (val_data, val_labels),
    epochs=100,
    batch_size=16,
    callbacks=[callbacks],
)



In [ ]:
predictions = seq_model.predict(x=val_data, batch_size=16, verbose=1)

In [ ]:
rounded_predictions = np.argmax(predictions, axis=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
cm = confusion_matrix(y_true = val_labels, y_pred = rounded_predictions)

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap = plt.cm.Blues):
    
    plt.imshow(cm, interpolation = 'nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print("Confusion matrix but withoout Normalization")
        
    print(cm)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = [ 'Food', 'Hello', 'HowAreYou', 'Internet', 'Like', 'Ready', 'Sad', 'Talk', 'ThankYou', 'Yes' ]
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='confusion matrix')